<a href="https://colab.research.google.com/github/jialubio/SiameseNetwork_BrachingPatterns/blob/main/Siamese_network_for_optimal_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Libraries

In [ ]:
# Up load necessary files
from google.colab import files
files.upload()

Using TensorFlow backend.


ImportError: ignored

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%tensorflow_version 1.x

!pip install git+https://github.com/raghakot/keras-vis.git -U
!pip install -I scipy==1.2.*

from vis.visualization import visualize_saliency,visualize_cam
from vis.utils import utils

import keras
from keras import activations,models
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten,Activation,Conv2D, MaxPooling2D
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1.keras import backend
print(tf.__version__)

import numpy as np
import pandas as pd
from PIL import ImageOps,Image
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
import seaborn as sn
import os
import math
import sys
from tqdm import tqdm
from utils_saliency_map import special_csv_reader,label_str_to_dec, label_dec_to_binary,\
     label_binary_to_dec, class_counter,data_distribution,image_set,\
     train_valid_loss_plot,train_valid_acc_plot,pred_standard,save_saliency_map,\
     save_saliency_map,create_circular_mask

# 1. Upload files

In [ ]:
# Unzip dataset
!unzip -q '/content/gdrive/My Drive/' \
-d '/content/dataset'

In [ ]:
# Path
path = "/content/dataset/"
path_final = path + "final/"
path_plaintext = path
path_out = path + "/output/"
!mkdir "/content/dataset/output"

filename = path_plaintext + 'labels.csv'

In [ ]:
char_list = range(1,4)
char_list = [str(i) for i in char_list]
int2char  = dict(enumerate(char_list))
char2int  = {char: ind for ind, char in int2char.items()}

num_classes = len(char_list)
num_replicate = 1000
dataset_size = num_classes*num_replicate

# 2.Data preprocessing

In [ ]:
# Labels
# Convert alphabetic labels into decimal and binary
y = []
with open(filename,"r") as csvFile:
    for row in csvFile:
        y.append(row[:-1])

y = np.asarray(y)
y_dec = label_str_to_dec(y[0:dataset_size],char2int).reshape(dataset_size,1)
y_binary = label_dec_to_binary(y_dec[0:dataset_size],num_classes)

In [ ]:
# Load images and resize
ResizedHeight = 80
train_data = image_set(path_final,'FImg_ID_',
                       dataset_size,
                       resize=True,
                       height=ResizedHeight,
                       width=ResizedHeight)

# 3. Create dataset

## Construct image pairs

In [ ]:
def get_random_image(img_groups, group_names, gid):
    gname = group_names[gid]
    photos = img_groups[gname]
    pid = np.random.choice(np.arange(len(photos)), size=1)[0]
    pname = photos[pid]
    return gname + pname + ".jpg"

In [ ]:
def create_triples(image_dir):
    img_groups = {}
    for img_file in os.listdir(image_dir):
        prefix, suffix = img_file.split(".")
        gid, pid = prefix[0:4], prefix[4:]
        if img_groups.has_key(gid):
            img_groups[gid].append(pid)
        else:
            img_groups[gid] = [pid]

    pos_triples, neg_triples = [], []

    # positive pairs are any combination of images in same group
    for key in img_groups.keys():
        triples = [(key + x[0] + ".jpg", key + x[1] + ".jpg", 1)
                 for x in itertools.combinations(img_groups[key], 2)]
        pos_triples.extend(triples)

    # need equal number of negative examples
    group_names = list(img_groups.keys())
    for i in range(len(pos_triples)):
        g1, g2 = np.random.choice(np.arange(len(group_names)), size=2, replace=False)
        left = get_random_image(img_groups, group_names, g1)
        right = get_random_image(img_groups, group_names, g2)
        neg_triples.append((left, right, 0))

    pos_triples.extend(neg_triples)
    shuffle(pos_triples)

    return pos_triples

In [ ]:
def make_img_pairs(n,num_classes,y):
  # output: [(img_idx_1, img_idx_2, label)]
  # img_idx: index of images in the original image stack
  # label: 1 - same class, 0 - different class

  # get image indices of each class
  img_classes = {}
  for i in num_classes:
    indices = np.where(y[:, i] == 1.)[i]
    img_classes[i] = indices

  sim_img_pairs,dis_img_pairs = [],[]

  # make similar image pairs
  for key in img_classes.keys():
    new_pair = [(x[0], x[1], 1) \
               for x in itertools.combinations(img_classes[key], 2)]

    sim_img_pairs.extend(new_pair)

  # make dissimilar image pairs
  class_names = list(img_classes.keys())
    for i in range(len(pos_triples)):
        class1, class2 = np.random.choice(np.arange(num_classes), size=2, replace=False) + 1
        idx1 = np.random.choice(np.arange(len(img_classes[class1])))
        idx2 = np.random.choice(np.arange(len(img_classes[class2])))
        left = img_classes(class1)[idx1]
        right = img_classes(class2)[idx2]
        dis_img_pairs.append((left, right, 0))

  # Combine and shuffle the image pairs
  sim_img_pairs.extend(dis_img_pairs)
  shuffle(sim_img_pairs)
  return img_pairs

In [ ]:
img_classes = {1:(1,2,3,6,7,9), 2:(1,2,3,6,7,9)}
num_classes = 2
class1, class2 = np.random.choice(np.arange(num_classes), size=2, replace=False) +1



idx1 = np.random.choice(np.arange(l1))
idx2 = np.random.choice(np.arange(l2))
print(idx1,idx2)



2 4


In [ ]:
img_classes.get(class1)
len(img_classes.get(class1))


TypeError: ignored

In [ ]:
triples_data = create_triples(IMAGE_DIR)

print("# image triples:", len(triples_data))
[x for x in triples_data[0:5]]

## Split dataset

In [1]:
# Split training, test and validation sets
X_data, X_test, y_data, y_test = train_test_split(train_data.image_stack, y_binary,stratify=y_binary,test_size=0.1,shuffle=True, random_state=25)
if split_idx == 0:
    X_train, X_val, y_train, y_val = train_test_split(X_data,
                                                      y_data,
                                                      stratify=y_data,
                                                      test_size=0.11111,
                                                      shuffle=True,
                                                      random_state=25)
else:
    X_data, X_val, y_data, y_val = train_test_split(X_data,
                                                    y_data,
                                                    stratify=y_data,
                                                    test_size=0.11111,
                                                    shuffle=True,
                                                    random_state=25)
    seg_ratio = [0.5,0.75,0.875,0.9375,0.96875,0.9875,0.9975]
    X_train, X_left, y_train, y_left = train_test_split(X_data,
                                                        y_data,
                                                        stratify=y_data,
                                                        test_size=seg_ratio[split_idx-1],
                                                        shuffle=True,
                                                        random_state=25)

trainsize = X_train.shape[0]

NameError: name 'train_test_split' is not defined

In [ ]:
# Reshape dataset
X_train = X_train.reshape((len(X_train),ResizedHeight,ResizedHeight,1))
X_test = X_test.reshape((len(X_test),ResizedHeight,ResizedHeight,1))
X_val = X_val.reshape((len(X_val),ResizedHeight,ResizedHeight,1))
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

# 4. Siamese network

In [ ]:
# Build model
initializer=keras.initializers.glorot_normal(seed=27) # change the seed
model = keras.Sequential()
# model.add(keras.layers.Reshape((ResizedHeight,ResizedHeight,1), input_shape=(ResizedHeight,ResizedHeight,)))

model.add(Conv2D(64, kernel_size=(5, 5),padding='same',kernel_initializer=initializer)) #3/5/7 ,activation='relu'
model.add(keras.layers.MaxPooling2D(pool_size=(10, 10))) # 40,2
model.add(Activation('relu'))

model.add(Conv2D(128, kernel_size=(5, 5),padding='same',kernel_initializer=initializer)) #3/5/7 ,activation='relu'
model.add(keras.layers.MaxPooling2D(pool_size=(8, 8))) # 40,2
model.add(Activation('relu'))

model.add(keras.layers.Reshape((128*1*1,), input_shape=(1,1,128)))
model.add(Dense(50, activation='relu',kernel_initializer=initializer))
model.add(Dense(num_classes, activation='softmax',kernel_initializer=initializer))

# Compile model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.001, nesterov=False),
              metrics=['accuracy'])

In [ ]:
earlystopping_loss = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        min_delta=0.00001,
                                                        patience=30,
                                                        verbose=0,
                                                        mode='auto',
                                                        baseline=None,
                                                        restore_best_weights=True)

In [ ]:
# Train with early stopping
for i in range(0, 1):
    history = model.fit(X_train,
                      y_train,
                      epochs = 50,
                      batch_size=16,
                      verbose=1,
                      validation_data=(X_val, y_val),
                      callbacks=[earlystopping_loss])

    a = bool( earlystopping_loss.stopped_epoch >= 30 and earlystopping_loss.stopped_epoch < 100)
    if i == 0:
        stopped_epoch = [earlystopping_loss.stopped_epoch]
    else:
        next_epoch    = [earlystopping_loss.stopped_epoch]
        stopped_epoch = np.concatenate((stopped_epoch,next_epoch), axis = 0)
    if a:
        break

# 5. Predict similarity score

In [ ]:
# Predict
pred_y_train = pred_standard(model2.predict(X_train, batch_size=None, verbose=0, steps=None))
pred_y_test  = pred_standard(model2.predict(X_test,  batch_size=None, verbose=0, steps=None))
pred_y_val   = pred_standard(model2.predict(X_val,   batch_size=None, verbose=0, steps=None))

# Compute and save accuracy
acc_train = accuracy_score(y_train, pred_y_train)
acc_test  = accuracy_score(y_test, pred_y_test)
acc_val   = accuracy_score(y_val, pred_y_val)
accuracy_list = [acc_train,acc_val,acc_test]
print(accuracy_list)

#filename4 = path_out + Simulation_Parameters + "class_" + str(num_classes) + "_trainsize_" + str(trainsize) + "_accuracy" + ".txt"
#np.savetxt(filename4, accuracy_list, delimiter=',', fmt = '%f')